In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.plotting.backend = 'plotly'   # Configura os gráficos como plotly

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Exploration

In [ ]:
data = pd.read_csv('../input/tweet-sentiment-extraction/train.csv')
data

In [ ]:
data.info()

In [ ]:
print(f'Os dados possuem {data.shape[0]} linhas e {data.shape[1]} colunas')
print(f'Os dados possuem {data.isna().sum().sum()} valores nulos')

In [ ]:
data[['sentiment']]

In [ ]:
sentiments = data.sentiment.str.split(expand=True).stack().value_counts()
sentiments

In [ ]:
data.sentiment.hist(xrot='Sentiment', yrot='Quantity')

# Separação dos Sentimentos

In [ ]:
# Seleção dos dados classificados como neutros
selecao_neutros = data['sentiment'] == 'neutral'
neutros = data[selecao_neutros]
neutros

In [ ]:
# Seleção dos dados classificados como negativos
negativos = data.query('sentiment == "negative"')
negativos

In [ ]:
# Seleção dos dados classificados como positivos
selecao_positivos = data['sentiment'] == 'positive'
positivos = data[selecao_positivos]
positivos

# Tratamento de Texto Categórico

In [ ]:
data_ref = data.copy()

In [ ]:
data_ref = data_ref.replace(',', ' ', regex=True)

In [ ]:
from sklearn import preprocessing

In [ ]:
ohe = preprocessing.OneHotEncoder(handle_unknown='ignore', sparse=False)
ohe.fit(data_ref[['sentiment']])

In [ ]:
ohe.get_feature_names(['sentiment'])

In [ ]:
# Retorna um Array (Matriz) com uma coluna numérica para cada sentimento
X = ohe.transform(data_ref[['sentiment']])

data_numerico = pd.DataFrame(X, columns=ohe.get_feature_names(['sentiment']))
data_numerico

### Deletando a Coluna Texto

In [ ]:
data_ref = data_ref.drop(['sentiment'], axis=1)
data_ref

### Concatenando os DataFrames

In [ ]:
data_proc = pd.concat([data_ref, data_numerico], axis=1)
data_proc

# Processamento de Texto Semântico

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vec_selected_text = TfidfVectorizer(strip_accents='ascii')

X_selected_text = vec_selected_text.fit_transform(data_proc['selected_text'].values.astype('str')).todense()

df_selected_text = pd.DataFrame(X_selected_text, columns=vec_selected_text.get_feature_names())

In [ ]:
data_proc.shape

In [ ]:
df_selected_text

In [ ]:
df_selected_text.loc[df_selected_text['zoozoo'] > 0]

In [ ]:
df_selected_text.loc[df_selected_text['000th'] > 0]

In [ ]:
df_selected_text.dtypes

# Calculando a Pontuação

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
def acerto(dados, coluna_teste, coluna_classificacao):
    vetorizar = CountVectorizer(lowercase=True, max_features=50)
    bag_of_words = vetorizar.fit_transform(dados[coluna_teste])
    
    X_train, X_test, y_train, y_test = train_test_split(bag_of_words, dados[coluna_classificacao], random_state=50)
    
    regressao_logistica = LogisticRegression()
    regressao_logistica.fit(X_train, y_train)
    return regressao_logistica.score(X_test, y_test)

In [ ]:
data_proc['text'] = data_proc['text'].astype('str')
data_proc.dtypes

In [ ]:
acerto(data_proc, 'text', 'sentiment_negative')

# Trabalhando com WordCloud

In [ ]:
from wordcloud import wordcloud, WordCloud
import matplotlib.pyplot as plt

In [ ]:
todas_palavras = [texto for texto in data_proc['text']]
todas_palavras = ''.join(str(todas_palavras))

In [ ]:
todas_palavras[0:50]

In [ ]:
nuvem_palavras = WordCloud(width=800, height=500, max_font_size=110, collocations=False).generate(todas_palavras)

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(nuvem_palavras, interpolation='bilinear')
plt.axis('off')
plt.show()

## Função para nuvem de palavras negativas

In [ ]:
def nuvem_palavras_neg(dados, coluna):
    texto_negativo = dados.query('sentiment_negative == 1')
    todas_palavras = ''.join([texto for texto in texto_negativo[coluna]])
    
    nuvem_palavras = WordCloud(width=800, height=500, max_font_size=110, collocations=False).generate(todas_palavras)

    plt.figure(figsize=(12,10))
    plt.imshow(nuvem_palavras, interpolation='bilinear')
    plt.axis('off')
    plt.show()

## Função para nuvem de palavras negativas

In [ ]:
def nuvem_palavras_pos(dados, coluna):
    texto_positivo = dados.query('sentiment_positive == 1')
    todas_palavras = ''.join([texto for texto in texto_positivo[coluna]])
    
    nuvem_palavras = WordCloud(width=800, height=500, max_font_size=110, collocations=False).generate(todas_palavras)

    plt.figure(figsize=(12,10))
    plt.imshow(nuvem_palavras, interpolation='bilinear')
    plt.axis('off')
    plt.show()

### WordCloud Negativa

In [ ]:
nuvem_palavras_neg(data_proc, 'text')

### WordCloud Positiva

In [ ]:
nuvem_palavras_pos(data_proc, 'text')

# Frequência das Palavras

In [ ]:
from nltk import tokenize
import nltk

In [ ]:
token_espaco = tokenize.WhitespaceTokenizer()
token_palavras = token_espaco.tokenize(todas_palavras)
frequencia = nltk.FreqDist(token_palavras)

In [ ]:
df_frequencia = pd.DataFrame({'Palavra' : list(frequencia.keys()), 'Frequência' : list(frequencia.values())})
df_frequencia

In [ ]:
import seaborn as sns

In [ ]:
def grafico(dados, coluna, quantidade):
    todas_palavras = ''.join(str([dado for dado in dados[coluna]]))
    token_palavras = token_espaco.tokenize(todas_palavras)
    frequencia = nltk.FreqDist(token_palavras)
    df_frequencia = pd.DataFrame({'Palavra' : list(frequencia.keys()), 'Frequência' : list(frequencia.values())})
    df_frequencia = df_frequencia.nlargest(columns = 'Frequência', n = quantidade)
    
    
    plt.figure(figsize=(12, 10))
    ax = sns.barplot(data = df_frequencia, x = 'Palavra', y = 'Frequência', color = 'crimson')
    ax.set(ylabel = 'Contagem')
    plt.show()

In [ ]:
grafico(data_proc, 'text', 10)

# Remoção de Palavras Irrelevantes

In [ ]:
palavras_irrelevantes = nltk.corpus.stopwords.words('english')
palavras_irrelevantes

In [ ]:
frase_processada = list()

for opiniao in data_proc.text:
    nova_frase = list()
    palavras_texto = token_espaco.tokenize(str(opiniao))
    for palavra in palavras_texto:
        if palavra not in palavras_irrelevantes:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))
    
data_proc['tratamento_1'] = frase_processada

In [ ]:
data_proc.head()

In [ ]:
print(f"Sem tratamento: {acerto(data_proc, 'text', 'sentiment_negative')}\n")
print(f"Primeiro tratamento: {acerto(data_proc, 'tratamento_1', 'sentiment_negative')}")

# Remoção da Pontuação

In [ ]:
from string import punctuation

In [ ]:
punctuation

In [ ]:
pontuacao = list()
for ponto in punctuation:
    pontuacao.append(ponto)
    
pontuacao.append('...')
pontuacao.append('..')
pontuacao.append('://')
pontuacao

### Adicionando as pontuações na lista de StopWords

In [ ]:
pontuacao_stopwords = pontuacao + palavras_irrelevantes

In [ ]:
token_pontuacao = tokenize.WordPunctTokenizer()

In [ ]:
frase_processada = list()

for opiniao in data_proc['tratamento_1']:
    nova_frase = list()
    palavras_texto = token_pontuacao.tokenize(opiniao)
    for palavra in palavras_texto:
        if palavra not in pontuacao_stopwords:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))
    
data_proc['tratamento_2'] = frase_processada

In [ ]:
data_proc.head()

# Transformação do Texto em Minúsculo

In [ ]:
frase_processada = list()

for opiniao in data_proc['tratamento_2']:
    nova_frase = list()
    opiniao = opiniao.lower()
    palavras_texto = token_pontuacao.tokenize(opiniao)
    for palavra in palavras_texto:
        if palavra not in pontuacao_stopwords:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))
    
data_proc['tratamento_3'] = frase_processada

In [ ]:
data_proc.head()

In [ ]:
acuracia_tratamento_3 = acerto(data_proc, 'tratamento_3', 'sentiment_negative')
acuracia_tratamento_2 = acerto(data_proc, 'tratamento_2', 'sentiment_negative')

print(f'Acurácia 2: {acuracia_tratamento_2}')
print(f'Acurácia 3: {acuracia_tratamento_3}')

## Nuvem de Palavras Negativas do Tratamento 3

In [ ]:
nuvem_palavras_neg(data_proc, 'tratamento_3')

## Nuvem de Palavras Positivas do Tratamento 3

In [ ]:
nuvem_palavras_pos(data_proc, 'tratamento_3')

## Gráfico das Palavras mais utilizadas no Tratamento 3

In [ ]:
grafico(data_proc, 'tratamento_3', 10)

# Ngrams

[ngrams](https://kavita-ganesan.com/what-are-n-grams/#.YMANDZpKjUQ)

In [ ]:
from nltk import ngrams

In [ ]:
tfidf = TfidfVectorizer(lowercase=False, ngram_range=(1, 2))
vetor_tfidf = tfidf.fit_transform(data_proc['tratamento_3'])
X_train, X_test, y_train, y_test = train_test_split(vetor_tfidf, data_proc['sentiment_negative'], random_state=50)

regressao_logistica = LogisticRegression()
regressao_logistica.fit(X_train, y_train)
acuracia_tfidf_ngrams = regressao_logistica.score(X_test, y_test)

print(f'Ngrams: {acuracia_tfidf_ngrams}\n')
print(f'Tratamento 3: {acuracia_tratamento_3}')